<a href="https://colab.research.google.com/github/sachinbabuantony/AOG/blob/main/AOG_Email_Syst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install requests

In [1]:
import requests
import pandas as pd
from flask import Flask, request, url_for
from datetime import datetime, timedelta
import pytz

In [2]:
data = {
    'Email Id': ['olimpio@aviationnews-online.com', 'Dino@email2.com', 'sachinbabuantony@gmail.com', 'chrisb@aviationnews----online.com'],
    'Flight Serial No': [["34923", "5507", "3758"], ['126', '5507', '096'], ['126', '5507', '096'], [ '126', '5507', '096']],
    'AOG threshold(Days)': [2, 1, 3, 1]
}

clients_df = pd.DataFrame(data)
clients_df

,Email Id,Flight Serial No,AOG threshold(Days)
0,olimpio@aviationnews-online.com,"[34923, 5507, 3758]",2
1,Dino@email2.com,"[126, 5507, 096]",1
2,sachinbabuantony@gmail.com,"[126, 5507, 096]",3
3,chrisb@aviationnews----online.com,"[126, 5507, 096]",1


In [3]:
serial_numbers = []
for index in clients_df.index:
    for serial in clients_df['Flight Serial No'][index]:
        if serial not in serial_numbers:
            serial_numbers.append(serial)
#["4201","34923","7488","3758"],
serial_numbers

['34923', '5507', '3758', '126', '096']

In [8]:
# Token for authentication
API_TOKEN = 'f817516b1241f95db82006cd603ee62dd50732b1'  # Replace with your actual API token
pd.set_option("display.max_columns", None)
# URL for the API endpoint
url = 'https://api.radarbox.com/v2/flights/search'

current_time = datetime.utcnow()

# Get the time 7 days ago from now
seven_days_ago = current_time - timedelta(days=4)

# Convert to ISO 8601 format (used in APIs)
to_date = current_time.strftime("%Y-%m-%dT%H:%M:%SZ")
from_date = seven_days_ago.strftime("%Y-%m-%dT%H:%M:%SZ")

# Calculate the page size (1 + total number of registrations)
page_size = 5

# Request schema including date restrictions and registrations
payload = {
    "pageSize": page_size,
    "page": 1,
    "toDate": to_date,
    "fromDate": from_date,
    "serialNumbers": ["36292", "40127", "12152", "36280", "19000130"],
    "aircraftClasses": [
    "AIRLINER"
  ]
}

# Headers for authorization and content type
headers = {
    'Authorization': f'Bearer {API_TOKEN}',
    'Content-Type': 'application/json'
}

# Sending POST request
response = requests.post(url, headers=headers, json=payload)

# Check if the request was successful
if response.status_code == 200:
    flight_data = response.json()
    # Check if 'flights' key is in the response
    if 'flights' in flight_data:
        df = pd.json_normalize(flight_data['flights'])
  # or display(df) if using Jupyter Notebook/Lab
    else:
        print("The 'flights' key is not found in the API response.")
        print("Response structure:", flight_data)  # Print response for debugging
else:
    print(f"Error fetching data: {response.status_code} - {response.text}")

In [9]:
df

,callsign,flightNumberIata,flightNumberIcao,aircraftRegistration,aircraftModeS,aircraftSerialNumber,aircraftType,aircraftClasses,aircraftTypeDescription,airlineIata,airlineIcao,airlineName,depAirportIcao,depAirportIata,depAirportName,depAirportCity,depAirportState,depAirportCountry,depAirportCountryIso2,depAirportCountryIso3,depAirportLatitude,depAirportLongitude,depAirportTz,depAirportTzDiffUTC,scheduledDeparture,actualTakeoff,calculatedTakeoff,arrAirportIcao,arrAirportIata,arrAirportName,arrAirportCity,arrAirportState,arrAirportCountry,arrAirportCountryIso2,arrAirportCountryIso3,arrAirportLatitude,arrAirportLongitude,arrAirportTz,arrAirportTzDiffUTC,scheduledArrival,estimatedArrival,actualLanding,calculatedLanding,departureStatus,departureDelayReason,departureDelayDetail,departureTerminal,arrivalStatus,arrivalDelayReason,arrivalDelayDetail,arrivalTerminal,arrivalBaggageClaim,latitude,longitude,squawkCode,source,created,updated,status,actualDeparture,departureGate,arrivalGate,altitude,codeshares,estimatedDeparture,icaoRoute,actualArrival
0,BBC347,BG347,BBC347,S2-AJT,702087,40127,B788,[AIRLINER],Boeing 787-8,BG,BBC,Biman Bangladesh Airlines,VGHS,DAC,Dhaka / Hazrat Shahjalal International Airport,Dhaka,Dhaka Division,Bangladesh,BD,BGD,23.8433,90.39780,BDT,6.0,2024-11-11T14:55:00.000+00:00,2024-11-11T15:16:00.000+00:00,2024-11-11T15:16:38.000+00:00,OMDB,DXB,Dubai International Airport,Dubai,Dubai Emirate,United Arab Emirates,AE,ARE,25.2528,55.36440,GST,4.0,2024-11-11T20:30:00.000+00:00,2024-11-11T20:13:00.000+00:00,2024-11-11T20:11:00.000+00:00,2024-11-11T20:09:27.000+00:00,NS,6,OFF,2,NS,6,ON,1,6,25.258987,55.345207,4005,ADSB,2024-11-11T14:55:54.000+00:00,2024-11-11T20:44:43.615+00:00,LANDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AIC306,AI306,AIC306,VT-ANE,8005EF,36280,B788,[AIRLINER],Boeing 787-8,AI,AIC,Air India,VIDP,DEL,Indira Gandhi International Airport,New Delhi,Delhi,India,IN,IND,28.5665,77.10310,IST,5.5,2024-11-11T18:45:00.000+00:00,2024-11-11T19:33:00.000+00:00,2024-11-11T19:33:29.000+00:00,RJAA,NRT,Narita International Airport,Tokyo Narita,Chiba Prefecture,Japan,JP,JPN,35.7647,140.38600,JST,9.0,2024-11-12T02:00:00.000+00:00,2024-11-12T02:28:00.000+00:00,NaN,NaN,DY,6,OFF,3,NS,6,EON,2,NaN,30.343630,113.590010,546,ADSB,2024-11-11T19:14:14.000+00:00,2024-11-11T23:39:14.000+00:00,IN_FLIGHT,2024-11-11T19:14:00.000+00:00,14,72,39100.0,NH6890,NaN,NaN,NaN
2,AIC120,AI120,AIC120,VT-ANU,80073B,36292,B788,[AIRLINER],Boeing 787-8,AI,AIC,Air India,EDDF,FRA,Frankfurt am Main International Airport,Frankfurt,Hesse,Germany,DE,DEU,50.0264,8.54313,CET,1.0,2024-11-11T19:00:00.000+00:00,2024-11-11T19:41:00.000+00:00,2024-11-11T19:41:25.000+00:00,VIDP,DEL,Indira Gandhi International Airport,New Delhi,Delhi,India,IN,IND,28.5665,77.10310,IST,5.5,2024-11-12T02:40:00.000+00:00,2024-11-12T02:34:00.000+00:00,NaN,NaN,DY,6,OFF,1,NS,6,EON,3,11,39.889542,50.180847,7712,ADSB,2024-11-11T19:14:23.000+00:00,2024-11-11T23:39:56.000+00:00,IN_FLIGHT,2024-11-11T19:23:00.000+00:00,C13,NaN,38975.0,LH7252,2024-11-11T19:00:00.000+00:00,N0463F310 SULUS2D SULUS Z650 TONSU/N0488F350 Z...,NaN
3,KLM967,KL967,KLM967,G-CLSN,407B04,19000130,E190,[AIRLINER],Embraer 190-100LR,KL,KLM,KLM Royal Dutch Airlines,EHAM,AMS,Amsterdam Airport Schiphol,Amsterdam,Noord-Holland,Netherlands,NL,NLD,52.3086,4.76389,CET,1.0,2024-11-11T20:20:00.000+00:00,2024-11-11T20:52:00.000+00:00,2024-11-11T20:53:07.000+00:00,EGNV,MME,Teesside International Airport,Durham,England,United Kingdom,GB,GBR,54.5092,-1.42941,GMT,0.0,2024-11-11T21:35:00.000+00:00,2024-11-11T21:52:00.000+00:00,2024-11-11T21:42:00.000+00:00,2024-11-11T21:41:18.000+00:00,DY,2,OFF,NaN,DY,6,IN,NaN,NaN,54.507648,-1.437674,7051,ADSB,2024-11-11T20:30:25.000+00:00,2024-11-11T21:56:48.029+00:00,LANDED,2024-11-11T20:31:00.000+00:00,D29,NaN,NaN,"VS7090,KQ1537,AZ3753,AF8357,EY7347",2024-11-11T20:20:00.000+00:00,N0459F320 BERGI L602 NALAX L46 ODNEK N110 ERKI...,2024-11-11T21:46:00.000+00:00
4,BBC148,BG148,BBC148,S2-AJT,702087,40127,B788,[AIRLINER

In [10]:
selected_columns = ['aircraftSerialNumber', 'aircraftRegistration', 'airlineName', 'actualTakeoff', 'actualLanding', 'aircraftTypeDescription', 'updated', 'status', 'arrAirportCity', 'depAirportName']
selected_df = df.reindex(columns=selected_columns)  # Change made here
# # Drop rows with NaN values in 'actualTakeoff' before grouping
# selected_df = selected_df.dropna(subset=['actualTakeoff'])
selected_df

,aircraftSerialNumber,aircraftRegistration,airlineName,actualTakeoff,actualLanding,aircraftTypeDescription,updated,status,arrAirportCity,depAirportName
0,40127,S2-AJT,Biman Bangladesh Airlines,2024-11-11T15:16:00.000+00:00,2024-11-11T20:11:00.000+00:00,Boeing 787-8,2024-11-11T20:44:43.615+00:00,LANDED,Dubai,Dhaka / Hazrat Shahjalal International Airport
1,36280,VT-ANE,Air India,2024-11-11T19:33:00.000+00:00,NaN,Boeing 787-8,2024-11-11T23:39:14.000+00:00,IN_FLIGHT,Tokyo Narita,Indira Gandhi International Airport
2,36292,VT-ANU,Air India,2024-11-11T19:41:00.000+00:00,NaN,Boeing 787-8,2024-11-11T23:39:56.000+00:00,IN_FLIGHT,New Delhi,Frankfurt am Main International Airport
3,19000130,G-CLSN,KLM Royal Dutch Airlines,2024-11-11T20:52:00.000+00:00,2024-11-11T21:42:00.000+00:00,Embraer 190-100LR,2024-11-11T21:56:48.029+00:00,LANDED,Durham,Amsterdam Airport Schiphol
4,40127,S2-AJT,Biman Bangladesh Airlines,NaN,NaN,Boeing 787-8,2024-11-11T23:39:51.441+00:00,IN_FLIGHT,Chittagong,Dubai International Airport


In [39]:
# Get all rows where 'status' is 'IN_FLIGHT'
inflight_df = selected_df[selected_df['status'] == 'IN_FLIGHT']

# Get aircraftSerialNumbers that are 'IN_FLIGHT'
inflight_serials = inflight_df['aircraftSerialNumber'].unique()

# For aircraft not in 'IN_FLIGHT', get the latest 'actualTakeoff' per 'aircraftSerialNumber'
not_inflight_df = selected_df[~selected_df['aircraftSerialNumber'].isin(inflight_serials)]
if not not_inflight_df.empty:
    # Drop rows with NaN values in 'actualTakeoff' before grouping
    not_inflight_df = not_inflight_df.dropna(subset=['actualTakeoff'])
    latest_not_inflight_df = not_inflight_df.loc[
        not_inflight_df.groupby('aircraftSerialNumber')['actualTakeoff'].idxmax()
    ]
    # Combine inflight_df and latest_not_inflight_df
    latest_takeoff_df = pd.concat([inflight_df, latest_not_inflight_df])
else:
    latest_takeoff_df = inflight_df

latest_takeoff_df

,aircraftSerialNumber,aircraftRegistration,airlineName,actualTakeoff,actualLanding,aircraftTypeDescription,updated,status,arrAirportCity,depAirportName
0,36292,VT-ANU,Air India,2024-11-11T08:42:00.000+00:00,NaN,Boeing 787-8,2024-11-11T16:11:57.000+00:00,IN_FLIGHT,Frankfurt,Indira Gandhi International Airport
1,12152,VT-IWM,IndiGo,2024-11-11T14:01:00.000+00:00,NaN,Airbus A321-251NX,2024-11-11T16:07:56.000+00:00,IN_FLIGHT,New Delhi,Kempegowda International Airport
3,40127,S2-AJT,Biman Bangladesh Airlines,2024-11-11T15:16:00.000+00:00,NaN,Boeing 787-8,2024-11-11T16:07:50.000+00:00,IN_FLIGHT,Dubai,Dhaka / Hazrat Shahjalal International Airport
2,19000130,G-CLSN,KLM Royal Dutch Airlines,2024-11-11T14:19:00.000+00:00,2024-11-11T15:19:00.000+00:00,Embraer 190-100LR,2024-11-11T15:39:10.980+00:00,LANDED,Amsterdam,Bristol International Airport


In [40]:
import pandas as pd
from datetime import datetime
import pytz

# Assuming latest_takeoff_df is your DataFrame that has already been grouped by aircraftSerialNumber
# Ensure latest_takeoff_df is a deep copy to avoid SettingWithCopyWarning
latest_takeoff_df = latest_takeoff_df.copy()

# Convert actualTakeoff and actualLanding to datetime format (if not already done)
latest_takeoff_df['actualTakeoff'] = pd.to_datetime(latest_takeoff_df['actualTakeoff'], errors='coerce')
latest_takeoff_df['actualLanding'] = pd.to_datetime(latest_takeoff_df['actualLanding'], errors='coerce')

# Create a new column to store AOG time in days
latest_takeoff_df['AOG_Till_Date_Days'] = None

# Current time in UTC
current_time_utc = datetime.utcnow().replace(tzinfo=pytz.utc)

# Apply AOG logic to each row in the latest_takeoff_df
for idx, row in latest_takeoff_df.iterrows():
    takeoff = row['actualTakeoff']
    landing = row['actualLanding']
    status = row['status']

    # New Logic: If the status is 'IN_FLIGHT', AOG_Till_Date_Days is zero
    if status == 'IN_FLIGHT':
        latest_takeoff_df.at[idx, 'AOG_Till_Date_Days'] = 0

    # Existing logic
    # Case 1: If the flight has taken off but hasn't landed, AOG is 0
    elif pd.notnull(takeoff) and pd.isnull(landing):
        latest_takeoff_df.at[idx, 'AOG_Till_Date_Days'] = 0

    # Case 2: If the flight has both taken off and landed, calculate AOG from last landing to current time in days
    elif pd.notnull(takeoff) and pd.notnull(landing):
        latest_takeoff_df.at[idx, 'AOG_Till_Date_Days'] = (current_time_utc - landing).total_seconds() / (3600 * 24)

# Display the DataFrame with the calculated AOG till date in days
latest_takeoff_df[['aircraftSerialNumber', 'AOG_Till_Date_Days']]

,aircraftSerialNumber,AOG_Till_Date_Days
0,36292,0
1,12152,0
3,40127,0
2,19000130,0.037278


In [35]:
import pandas as pd
import numpy as np

# Merging latest_takeoff_df with clients_df for aircraft registration check
result_list = []

for index, row in clients_df.iterrows():
    email = row['Email Id']
    threshold = row['AOG threshold(Days)']
    flight_serials = row['Flight Serial No']

    # Ensure threshold is a numeric type
    threshold = float(threshold)

    # Ensure flight_serials is a list
    if isinstance(flight_serials, str):
        # Assuming serial numbers are comma-separated
        flight_serials = [serial.strip() for serial in flight_serials.split(',')]
    elif not isinstance(flight_serials, list):
        flight_serials = [flight_serials]

    # Check if threshold is exceeded for each flight serial number
    for serial in flight_serials:
        if serial in latest_takeoff_df['aircraftSerialNumber'].values:
            # Get the AOG_Days and ensure it's numeric
            aog_days_series = latest_takeoff_df.loc[latest_takeoff_df['aircraftSerialNumber'] == serial, 'AOG_Till_Date_Days']
            if not aog_days_series.empty:
                aog_days = aog_days_series.values[0]
            else:
                aog_days = np.nan

            # Handle NaN or missing AOG_Days by assigning a default value
            if pd.isnull(aog_days):
                aog_days = 0.0  # Assign default value
            else:
                aog_days = float(aog_days)

            # Ensure comparison works correctly
            if aog_days > 0 and aog_days >= threshold:
                threshold_crossed = True
            else:
                threshold_crossed = False

            # Append the result to the list
            result_list.append({
                'Email Id': email,
                'Flight Serial No': serial,
                'AOG Days': aog_days,
                'Threshold Crossed': threshold_crossed
            })
        else:
            # Handle serial numbers not present in latest_takeoff_df
            # You may choose to include or exclude these entries
            aog_days = 0.0  # or assign np.nan
            threshold_crossed = False
            result_list.append({
                'Email Id': email,
                'Flight Serial No': serial,
                'AOG Days': aog_days,
                'Threshold Crossed': threshold_crossed
            })

# Convert to a DataFrame to view the results
results_df = pd.DataFrame(result_list)

# Ensure that 'AOG Days' is of numeric type
results_df['AOG Days'] = pd.to_numeric(results_df['AOG Days'], errors='coerce').fillna(0.0)

# Ensure 'Threshold Crossed' is boolean
results_df['Threshold Crossed'] = results_df['Threshold Crossed'].astype(bool)

# Display the results
results_df


,Email Id,Flight Serial No,AOG Days,Threshold Crossed
0,olimpio@aviationnews-online.com,34923,0.0,False
1,olimpio@aviationnews-online.com,5507,0.0,False
2,olimpio@aviationnews-online.com,3758,0.0,False
3,Dino@email2.com,126,0.0,False
4,Dino@email2.com,5507,0.0,False
5,Dino@email2.com,096,0.0,False
6,sachinbabuantony@gmail.com,126,0.0,False
7,sachinbabuantony@gmail.com,5507,0.0,False
8,sachinbabuantony@gmail.com,096,0.0,False
9,chrisb@aviationnews----online.com,126,0.0,False


In [17]:
# prompt: from results_df only display 'Threshold Crossed' = True

filtered_AOG = results_df[results_df['Threshold Crossed'] == True]

In [18]:
# Step 1: Import required libraries
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd

# Step 2: Set up the email details
sender_email = 'sachinbabuantony@gmail.com'   # Replace with your email address
sender_password = 'uwmz vflb bkie ddbo'       # Replace with your app-specific password

# Step 3: Define the function to send a single email with multiple flights per client
def send_email(receiver_email, flights_info):
    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = f'Flight AOG Alert: Few Flights Crossed Threshold'

    # Generate the table of flights and AOG hours
    flights_table = "<table border='1'><tr><th>Flight Serial No</th><th>AOG Hours</th></tr>"
    for flight_serial, aog_days in flights_info:
        flights_table += f"<tr><td>{flight_serial}</td><td>{aog_days}</td></tr>"
    flights_table += "</table>"

    # Email body with the table
    body = f"""\
    <html>
        <body>
            <p>Dear recipient,</p>
            <p>The following flights have exceeded the AOG threshold:</p>
            {flights_table}
            <p>Best Regards,<br>Airline Economics</p>
        </body>
    </html>
    """
    msg.attach(MIMEText(body, 'html'))

    try:
        # Set up the server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Secure the connection
        server.login(sender_email, sender_password)

        # Send the email
        server.send_message(msg)
        print(f"Email sent to {receiver_email} with details of {len(flights_info)} flights.")

        # Close the server connection
        server.quit()
    except Exception as e:
        print(f"Failed to send email to {receiver_email}: {e}")

# Step 4: Group the results by 'Email Id' and collect flight information for each client
grouped_clients = filtered_AOG.groupby('Email Id')

# Step 5: Iterate through the grouped data and send emails
for email, group in grouped_clients:
    # Collect flight serials and AOG hours into a list for the client
    flights_info = list(zip(group['Flight Serial No'], group['AOG Days']))
    send_email(email, flights_info)
